In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = sns.load_dataset('tips')

In [3]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [5]:
df.duplicated().sum()

1

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [8]:
df.corr()

/tmp/ipykernel_77/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,total_bill,tip,size
total_bill,1.000000,0.674998,0.597589
tip,0.674998,1.000000,0.488400
size,0.597589,0.488400,1.000000


In [9]:
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [10]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [11]:
num_cols = ['total_bill','tip','size']
cat_cols = ['sex','smoker','day']

In [12]:
X = df.drop('time',axis=1)
y = df['time']

In [13]:
X_train,X_test,y_train,y_test  = train_test_split(X,y,test_size=.25,random_state=42)

In [14]:
num_pipeline = Pipeline(
    steps = [('inpute',SimpleImputer(strategy='mean')),
             ('scaler',StandardScaler())
        ])
cat_pipline = Pipeline(
     steps=[('inpute',SimpleImputer(strategy='most_frequent')),
            ('onehotencoder',OneHotEncoder())
          
])

In [15]:
preprocessor= ColumnTransformer([
    ('num_pipline',num_pipeline,num_cols),
    ('col_pipline',cat_pipline,cat_cols)
])

In [16]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [17]:
models = {
    'Randomforest':RandomForestClassifier()
}


In [18]:
def eavluate(X_train,X_test,y_train,y_test,models):
    
    report = {}
    
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train,y_train)
        
        y_pred = model.predict(X_test)
        
        accuracy = accuracy_score(y_pred,y_test)
        report[list(models.keys())[i]] = accuracy
        
    return report 

In [19]:
eavluate(X_train,X_test,y_train,y_test,models)

{'Randomforest': 0.9836065573770492}

In [30]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [22]:
df = sns.load_dataset('iris')

In [23]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [25]:
df['species'] = encoder.fit_transform(df['species'])

In [27]:
X = df.drop('species',axis=1)
y = df['species']

In [28]:
y

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: species, Length: 150, dtype: int64

In [29]:
X_train,X_test,y_trian,y_test = train_test_split(X,y,test_size=.25,random_state=42)

In [31]:
Random =  RandomForestClassifier()
log  = LogisticRegression()
r_pip = make_pipeline(StandardScaler(),Random)
l_pip = make_pipeline(StandardScaler(),log)
vot = VotingClassifier(estimators=[
    ('rf',r_pip),
    ('lf',l_pip)
],voting='soft')